In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Parking Lot Analysis Pipeline
> **Note**: This notebook is for reference and educational purposes only. Not intended for production use.  
> **Questions?** mateuswagner at google.com
## Overview
This notebook implements an automated parking lot analysis system using a two-stage approach:

1. **Image Annotation** - Uses Google Gemini's vision model to analyze parking lot aerial images and place magenta dots above cars and empty spaces
2. **Dot Detection** - Applies computer vision techniques (HSV color filtering, morphological operations) to detect and count the magenta markers
3. **Visualization** - Generates multi-panel visualizations showing detection results and summary statistics

## Workflow
```
Input Images → Gemini Processing → Annotated Images → OpenCV Detection → Count & Visualization
```

## Requirements
- Google Cloud Vertex AI access with Gemini API enabled
- Input images in `../demo-images/` directory
- Output saved to `../outputs/processed/`

## 1. Environment Setup

In [ ]:
# Install required packages (run once)
# - google-genai: Google Generative AI SDK for Gemini API access
# - opencv-python: Computer vision library for image processing
# - numpy: Numerical operations for array manipulation
# - matplotlib: Visualization library for plotting results
!pip install --upgrade google-genai opencv-python numpy matplotlib

In [ ]:
# --- Google Generative AI ---
from google import genai
from google.genai import types

# --- Standard Library ---
import base64
import os
import glob
from pathlib import Path
from datetime import datetime
import time

# --- Computer Vision & Data Processing ---
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Initialize the Gemini client using Vertex AI authentication
# Requires: GOOGLE_CLOUD_PROJECT environment variable or gcloud auth
client = genai.Client(vertexai=True)

## 2. Image Annotation with Gemini

This section sends parking lot aerial images to Google Gemini, which analyzes the scene and returns annotated images with magenta dots placed above:
- **Occupied spaces** (cars, trucks, vehicles)
- **Empty parking spaces**
- **Obstructions** (shopping carts, garbage containers)

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Directory paths
input_dir = Path("../demo-images")          # Source images to process
output_dir = Path("../outputs/processed")   # Destination for annotated images
output_dir.mkdir(parents=True, exist_ok=True)

# Generate unique timestamp for this batch run
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# =============================================================================
# GEMINI MODEL CONFIGURATION
# =============================================================================

# Model selection - using Gemini 3 Pro with image generation capability
model = "gemini-3-pro-image-preview"

# Prompt instructs Gemini to annotate parking spaces with magenta markers
prompt = """
Place a small magenta dot (255, 0, 255) above each car and each empty parking space within the parking lots boundaries.
Also consider spaces occupied by garbage containers and shopping carts.
"""

# Generation parameters for image output
generate_content_config = types.GenerateContentConfig(
    temperature=1,              # Controls randomness (1 = more creative)
    top_p=0.95,                 # Nucleus sampling threshold
    max_output_tokens=32768,    # Maximum response length
    response_modalities=["TEXT", "IMAGE"],  # Enable both text and image output
    
    # Disable safety filters for aerial imagery processing
    safety_settings=[
        types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
        types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
        types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
        types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF")
    ],
    
    # Output image configuration
    image_config=types.ImageConfig(
        aspect_ratio="1:1",             # Square output
        image_size="1K",                # 1024x1024 resolution
        output_mime_type="image/png",   # PNG format for lossless quality
    ),
)

# =============================================================================
# IMAGE COLLECTION
# =============================================================================

# Gather all supported image formats from input directory
image_files = (
    sorted(input_dir.glob("*.png")) + 
    sorted(input_dir.glob("*.jpg")) + 
    sorted(input_dir.glob("*.jpeg"))
)

print(f"Found {len(image_files)} images to process")
print(f"Batch timestamp: {timestamp}")
print(f"Output directory: {output_dir}\n")

# =============================================================================
# BATCH PROCESSING LOOP
# =============================================================================

for img_path in image_files:
    print(f"Processing: {img_path.name}")
    
    # Load image as binary data
    with open(img_path, "rb") as f:
        image_data = f.read()
    
    # Determine MIME type based on file extension
    mime_type = "image/png" if img_path.suffix.lower() == ".png" else "image/jpeg"
    
    # Construct multimodal request with image + text prompt
    msg_image = types.Part.from_bytes(data=image_data, mime_type=mime_type)
    msg_text = types.Part.from_text(text=prompt)
    
    contents = [
        types.Content(role="user", parts=[msg_image, msg_text]),
    ]
    
    # Send request to Gemini API
    response = client.models.generate_content(
        model=model,
        contents=contents,
        config=generate_content_config
    )
    
    # Extract and save the annotated image from response
    if response.candidates and response.candidates[0].content.parts:
        for part in response.candidates[0].content.parts:
            if hasattr(part, 'inline_data') and part.inline_data:
                # Save with timestamped filename to avoid overwrites
                output_filename = f"{img_path.stem}_processed_{timestamp}{img_path.suffix}"
                output_path = output_dir / output_filename
                
                with open(output_path, "wb") as f:
                    f.write(part.inline_data.data)
                print(f"  Saved: {output_path}")
                
            elif hasattr(part, 'text') and part.text:
                # Log any text responses from the model
                print(f"  Model response: {part.text}")
    
    print()
    
    # Rate limiting: 10-second delay between requests to avoid API throttling
    time.sleep(10)

## 3. Magenta Dot Detection

This section uses OpenCV to detect and count the magenta dots placed by Gemini. The detection pipeline:

1. **Color Space Conversion** - Convert BGR to HSV for robust color detection
2. **HSV Thresholding** - Isolate magenta pixels using narrow hue range (H: 145-165)
3. **Morphological Cleaning** - Remove noise with opening/closing operations
4. **Contour Analysis** - Find connected components and filter by area

In [ ]:
# Locate all processed images from the Gemini annotation step
# Uses the batch timestamp to match files from the current run
processed_images = (
    sorted(output_dir.glob(f"*_processed_{timestamp}*.png")) +
    sorted(output_dir.glob(f"*_processed_{timestamp}*.jpg"))
)

print(f"Found {len(processed_images)} annotated images to analyze")
print(f"Source directory: {output_dir}\n")

# Initialize storage for detection results across all images
all_results = []

In [ ]:
# =============================================================================
# DETECTION PARAMETERS
# =============================================================================

# HSV color range for magenta detection
# Magenta in HSV: Hue ~300 degrees = 150 in OpenCV's 0-180 scale
# Using narrow range to avoid false positives from similar colors
LOWER_MAGENTA = np.array([145, 150, 150])  # [Hue, Saturation, Value] minimum
UPPER_MAGENTA = np.array([165, 255, 255])  # [Hue, Saturation, Value] maximum

# Contour area thresholds (in pixels) to filter valid dots
MIN_DOT_AREA = 10   # Dots smaller than this are likely noise
MAX_DOT_AREA = 60   # Dots larger than this are likely artifacts

# Morphological kernel sizes for noise reduction
KERNEL_OPEN = np.ones((3, 3), np.uint8)   # For removing small noise (opening)
KERNEL_CLOSE = np.ones((5, 5), np.uint8)  # For filling small gaps (closing)

# =============================================================================
# DETECTION LOOP
# =============================================================================

for image_path in processed_images:
    print(f"Analyzing: {image_path.name}")
    print("-" * 50)
    
    # Load the annotated image
    img = cv2.imread(str(image_path))
    if img is None:
        print(f"  ERROR: Failed to load image")
        continue
    
    print(f"  Dimensions: {img.shape[1]}x{img.shape[0]} pixels")
    
    # --- Step 1: Convert to HSV color space ---
    # HSV provides better color separation than BGR for thresholding
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # --- Step 2: Create binary mask for magenta pixels ---
    mask = cv2.inRange(hsv, LOWER_MAGENTA, UPPER_MAGENTA)
    
    # --- Step 3: Apply morphological operations to clean the mask ---
    # Opening: Erode then dilate - removes small noise spots
    mask_cleaned = cv2.morphologyEx(mask, cv2.MORPH_OPEN, KERNEL_OPEN, iterations=1)
    # Closing: Dilate then erode - fills small holes in detected regions
    mask_cleaned = cv2.morphologyEx(mask_cleaned, cv2.MORPH_CLOSE, KERNEL_CLOSE, iterations=1)
    
    # --- Step 4: Find contours (connected components) ---
    contours, _ = cv2.findContours(mask_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # --- Step 5: Filter contours by area to identify valid dots ---
    valid_contours = [
        cnt for cnt in contours 
        if MIN_DOT_AREA <= cv2.contourArea(cnt) <= MAX_DOT_AREA
    ]
    
    dot_count = len(valid_contours)
    areas = [cv2.contourArea(cnt) for cnt in valid_contours]
    
    # --- Report results ---
    print(f"  Raw contours detected: {len(contours)}")
    print(f"  Valid dots (area {MIN_DOT_AREA}-{MAX_DOT_AREA}px): {dot_count}")
    
    if areas:
        print(f"  Dot area stats: mean={np.mean(areas):.1f}, std={np.std(areas):.1f}, "
              f"range=[{min(areas):.0f}, {max(areas):.0f}]")
    
    # Store results for visualization
    all_results.append({
        'filename': image_path.name,
        'image': img,
        'mask': mask_cleaned,
        'contours': valid_contours,
        'dot_count': dot_count,
        'areas': areas
    })
    
    print(f"  TOTAL PARKING SPACES: {dot_count}\n")

# =============================================================================
# SUMMARY
# =============================================================================
print("=" * 50)
print(f"Detection complete: {len(all_results)} images analyzed")
print("=" * 50)

## 4. Visualization and Results

Generate diagnostic visualizations showing:
- Original annotated image from Gemini
- Binary detection mask after HSV thresholding
- Detected dots highlighted with green contours
- Isolated magenta pixels for verification

In [ ]:
# =============================================================================
# GENERATE MULTI-PANEL VISUALIZATIONS
# =============================================================================

for result in all_results:
    # Unpack result data
    img = result['image']
    mask = result['mask']
    contours = result['contours']
    dot_count = result['dot_count']
    filename = result['filename']
    
    # Create 2x2 subplot figure
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(f"Detection Results: {filename}", fontsize=16, fontweight='bold')
    
    # --- Panel 1: Original annotated image ---
    axes[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[0, 0].set_title('Gemini-Annotated Image', fontsize=12)
    axes[0, 0].axis('off')
    
    # --- Panel 2: Binary detection mask ---
    axes[0, 1].imshow(mask, cmap='gray')
    axes[0, 1].set_title('Magenta Detection Mask (HSV Threshold)', fontsize=12)
    axes[0, 1].axis('off')
    
    # --- Panel 3: Detected dots with contour overlay ---
    detection_overlay = img.copy()
    cv2.drawContours(detection_overlay, contours, -1, (0, 255, 0), 2)  # Green contours
    cv2.putText(
        detection_overlay, 
        f'Detected: {dot_count} dots', 
        (10, 50), 
        cv2.FONT_HERSHEY_SIMPLEX, 
        1.5, 
        (0, 255, 0), 
        3
    )
    axes[1, 0].imshow(cv2.cvtColor(detection_overlay, cv2.COLOR_BGR2RGB))
    axes[1, 0].set_title(f'Detected Dots: {dot_count}', fontsize=12)
    axes[1, 0].axis('off')
    
    # --- Panel 4: Isolated magenta pixels ---
    isolated_magenta = cv2.bitwise_and(img, img, mask=mask)
    axes[1, 1].imshow(cv2.cvtColor(isolated_magenta, cv2.COLOR_BGR2RGB))
    axes[1, 1].set_title('Isolated Magenta Pixels', fontsize=12)
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    
    # Save visualization to output directory
    viz_filename = filename.replace('_processed_', '_detection_')
    viz_path = output_dir / viz_filename
    plt.savefig(viz_path, dpi=150, bbox_inches='tight')
    print(f"Saved visualization: {viz_path}")
    
    plt.show()

# =============================================================================
# FINAL SUMMARY REPORT
# =============================================================================

print("\n" + "=" * 60)
print("PARKING LOT ANALYSIS SUMMARY")
print("=" * 60)

total_dots = sum(r['dot_count'] for r in all_results)
print(f"\nTotal images analyzed: {len(all_results)}")
print(f"Total parking spaces detected: {total_dots}")

if all_results:
    avg_dots = total_dots / len(all_results)
    print(f"Average spaces per image: {avg_dots:.1f}")
    
    print("\nBreakdown by image:")
    print("-" * 40)
    for r in all_results:
        print(f"  {r['filename']}: {r['dot_count']} spaces")